In [12]:
import pandas as pd
import numpy as np
from scipy.special import logit

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

test_size = 0.2
seed = 0

train_df = pd.read_csv("project_train.csv")
rows_to_remove = [68, 84, 94]
train_df = train_df.drop(rows_to_remove).reset_index(drop=True)
vars_to_transform = ["danceability", "energy", "speechiness", "acousticness", "liveness", "valence"]
train_df[vars_to_transform] = train_df[vars_to_transform].transform(logit)

# Fit the scaler on the training data
scaler = StandardScaler()
train_df[vars_to_transform + ['loudness', 'tempo']] = scaler.fit_transform(train_df[vars_to_transform + ['loudness', 'tempo']])

# Read and preprocess the test data
test_df = pd.read_csv("project_test.csv")

# Apply the same logit transformation to the test data
test_df[vars_to_transform] = test_df[vars_to_transform].transform(logit)

# Transform the test data using the fitted scaler
test_df[vars_to_transform + ['loudness', 'tempo']] = scaler.transform(test_df[vars_to_transform + ['loudness', 'tempo']])


# Gradient Boosting model
clf_gbm = GradientBoostingClassifier(learning_rate=0.1, max_depth=3, n_estimators=125, random_state=0)
clf_gbm.fit(train_df.drop(columns=['Label']), train_df['Label'])  # Fit model to entire training data

# Predict on the transformed test data
response_gbm = clf_gbm.predict(test_df)
output_df = pd.DataFrame({'Predictions': response_gbm})

# Save predictions to a csv file
output_df.to_csv('predictions_gbm2.csv', index=False)